In [1]:
import numpy as np
import os, sys
import pickle, functools, operator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random #shuffle
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from netCDF4 import Dataset

def SODAdata(Xdata, Ydata, out):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')

    #time_step = 1 month                                               
    sst_1 = np.zeros((99,12,24,72))

    sst_1[:,:,:,:] = inp1.variables['sst'][1:,0:12,:,:]
    #(99,12,24,72)
    #1872-1970

    sst_2 = np.zeros((1188,24,72))

    for i in range(99):
        sst_2[i*12:(i+1)*12,:,:] = sst_1[i,:,:,:]
    #(1188,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_2 = sst_2[1:,:,:]

    sst_3 = np.zeros((1176,12,24,72))
    #滑窗
    for i in range(1176):
        sst_3[i:,:,:] = sst_2[i:i+12,:,:]
    #(1176,12,24,72)


    #channel = 2
    #trX = np.zeros((1176,12,24,72,2))
    #trX[:,:,:,:,0] = sst_3
    #trX[:,:,:,:,1] = t300_3
    #sst_3 = sst_3.swapaxes(1, 3)
    #sst_3 = sst_3.swapaxes(1, 2)
    trX = sst_3
    del sst_1,sst_2,sst_3
    #del t300_1,t300_2,t300_3

    #保存np数组
    #np.save("./SODAdata/SODA_trX_ts1_out.npy",trX)

    #label
    inpv2 = np.zeros((1200))
    for i in range(100):
        inpv2[i*12:(i+1)*12] = inp2.variables['pr'][i,:,0,0]
    #(1200)

    #out = 1
    trY = np.zeros((1176,out,1))
    #滑窗
    for i in range(1176):
        trY[i,:,0] = inpv2[i:i+out]   
    #(1176,out)
    trY_decoder_input = np.zeros((1176,24,1))
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]

    #np.save("./SODAdata/SODA_trY_ts1_out%s.npy"%out,trY) 
    #(1176,12,24,72,2)
    #(1176,out,1)
    return trX ,trY, trY_decoder_input

def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    #sst_33 = sst_33.swapaxes(1, 3)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

trX, trY, trY_decoder_input = SODAdata('SODA.input.36mon.1871_1970.nc',\
                                          'SODA.label.12mon.1873_1972.nc', 24)

testX, testY = GOSDAdata('GODAS.input.36mon.1980_2015.nc',\
                         'GODAS.label.12mon.1982_2017.nc', 24)

"""
indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]
"""


In [14]:
trX = trX.reshape((-1, 12, 1728))
#trY = trY.reshape((-1, 12, 1728))
testX = testX.reshape((-1, 12, 1728))
#testY = testY.reshape((-1, 12, 1728))

In [16]:
BUFFER_SIZE = trX.shape[0]
BATCH_SIZE = 64
steps_per_epoch = trX.shape[0]//BATCH_SIZE
#embedding_dim = 256
units = 1024
#vocab_inp_size = len(inp_lang.word_index)+1
#vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((trX, trY)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([64, 12, 1728]), TensorShape([64, 24, 1]))

In [12]:
#vgg16
input_cnn = Input(shape=(24,72,1))
x = layers.Conv2D(filters=30,kernel_size=(3,3),padding="same", activation="relu")(input_cnn)
x = layers.Conv2D(filters=30,kernel_size=(3,3),padding="same", activation="relu")(x)
x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = layers.Conv2D(filters=50, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.Conv2D(filters=50, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
"""
x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)
"""

#x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
#x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
#x = layers.Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(x)
#x = layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(x)

x = layers.Flatten()(x)
x = Dense(units=4096)(x)
#Dense(units=4096,activation="relu")
outputs_cnn = x
model_cnn = Model(input_cnn,outputs_cnn)
model_cnn.summary()
#plot_model(model_cnn, to_file='model_train.png', show_shapes=True, show_layer_names=True)

_input = Input(shape=(12, 24, 72))
_input1 = tf.reshape(_input, [-1, 24, 72])
x = model_cnn(_input1)
#x1 = model_cnn1(_input1)
output = tf.reshape(x, [-1,12, 4096])
#output1 = tf.reshape(x1, [-1,12, 50 * 18 * 6])
#output = tf.concat([output,output1],2)#(None, 12, 6480)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 24, 72, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 24, 72, 30)        300       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 72, 30)        8130      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 36, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 36, 50)        13550     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 36, 50)        22550     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 18, 50)        

In [10]:
class Encoder(tf.keras.Model):
  def __init__(self,  enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    #self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, x, hidden):
    #x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    return output, state

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units))

In [18]:
encoder = Encoder(units, BATCH_SIZE)

# 样本输入
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Encoder output shape: (batch size, sequence length, units) (64, 12, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [19]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, query, values):
    # 隐藏层的形状 == （批大小，隐藏层大小）
    # hidden_with_time_axis 的形状 == （批大小，1，隐藏层大小）
    # 这样做是为了执行加法以计算分数  
    hidden_with_time_axis = tf.expand_dims(query, 1)

    # 分数的形状 == （批大小，最大长度，1）
    # 我们在最后一个轴上得到 1， 因为我们把分数应用于 self.V
    # 在应用 self.V 之前，张量的形状是（批大小，最大长度，单位）
    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    # 注意力权重 （attention_weights） 的形状 == （批大小，最大长度，1）
    attention_weights = tf.nn.softmax(score, axis=1)

    # 上下文向量 （context_vector） 求和之后的形状 == （批大小，隐藏层大小）
    context_vector = attention_weights * values
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights

In [20]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 12, 1)


In [24]:
class Decoder(tf.keras.Model):
  def __init__(self, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    #self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(1)

    # 用于注意力
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, hidden, enc_output):
    # 编码器输出 （enc_output） 的形状 == （批大小，最大长度，隐藏层大小）
    context_vector, attention_weights = self.attention(hidden, enc_output)#(64, 1024)

    # x 在通过嵌入层后的形状 == （批大小，1，嵌入维度）
    #x = self.embedding(x)

    # x 在拼接 （concatenation） 后的形状 == （批大小，1，嵌入维度 + 隐藏层大小）
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # 将合并后的向量传送到 GRU
    output, state = self.gru(x)

    # 输出的形状 == （批大小 * 1，隐藏层大小）
    output = tf.reshape(output, (-1, output.shape[2]))

    # 输出的形状 == （批大小，vocab）
    x = self.fc(output)

    return x, state, attention_weights

In [44]:
decoder = Decoder(units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (64, 1)


In [31]:
optimizer = tf.keras.optimizers.Adam()

In [55]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.random.uniform((64, 1, 1))
        # 教师强制 - 将目标词作为下一个输入
        for t in range(1, targ.shape[1]):
            # 将编码器输出 （enc_output） 传送至解码器
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            loss += tf.keras.losses.sparse_categorical_crossentropy(targ[:, t], predictions)
            
            # 使用教师强制
            dec_input = tf.expand_dims(targ[:, t], 1)
        
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

In [56]:
EPOCHS = 10
import time

for epoch in range(EPOCHS):
    start = time.time()
    
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
            
    # 每 2 个周期（epoch），保存（检查点）一次模型
    #if (epoch + 1) % 2 == 0:
    #    checkpoint.save(file_prefix = checkpoint_prefix)
        
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



InvalidArgumentError:  Received a label value of -1 which is outside the valid range of [0, 1).  Label values: 0 0 0 0 0 1 1 -1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
	 [[node SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits (defined at <ipython-input-55-75f9d20edc14>:14) ]] [Op:__inference_train_step_47839]

Function call stack:
train_step
